In [ ]:
import earthaccess
import numpy as np
from datetime import datetime, timedelta
import requests
import tempfile
import matplotlib.pyplot as plt
import rioxarray as rxr
import opera_utils
import pandas as pd
import os
import psutil
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import warnings

ncpus = len(psutil.Process().cpu_affinity())    # number of available CPUs
print('number of available CPUs: ', ncpus)
ncpus = 10  # manually set nworkers

In [ ]:
# Authenticate with NASA Earthdata Login
auth = earthaccess.login(strategy="netrc")
req_session = auth.get_session()

# Define the date range
start_date = datetime(2016, 1, 1)
end_date = datetime(2025, 5, 21)       # Sanity check

threshold_snowcover = 40    # threshold of snow cover

# Define the CMR-STAC API endpoint and collection ID
stac_api_url = "https://cmr.earthdata.nasa.gov/stac/NSIDC_ECS"
collection_id = "C2015398723-NSIDC_ECS"  # This is the collection ID for MOD10CM

figure_dir = 'figures_snowcover'
os.makedirs(figure_dir, exist_ok=True)

output_geojson = 'blockout_snow_disp_s1.geojson'

In [ ]:
def get_snow_cover_data_filter(year, month, gdf_frame, threshold_snowcover=40):
    # Format the date string
    date_str = f"{year}-{month:02d}-01"
    end_date_str = (datetime(year, month, 1) + timedelta(days=32)).replace(day=1).strftime("%Y-%m-%d")
    
    # Search for granules
    results = earthaccess.search_data(
        short_name="MOD10CM",
        cloud_hosted=True,
        temporal=(date_str, end_date_str),
        bounding_box=(-180, -90, 180, 90)  # Global coverage
    )
    
    if results:
        # Get the first granule (should only be one per month)
        granule = results[0]
        urls = granule.data_links()
        
        if urls:
            url = urls[0]
            print(f"Streaming snow cover data for {year}-{month:02d}...")

            try:
                # Stream the content to a temporary file
                with tempfile.NamedTemporaryFile(delete=False, suffix='.hdf') as temp_file:
                    print(f"Downloading file from {url}")
                    with requests.get(url, stream=True) as response:
                        response.raise_for_status()  # Raise an exception for bad status codes
                        for chunk in response.iter_content(chunk_size=8192):
                            temp_file.write(chunk)
                    temp_file_path = temp_file.name
                
                print(f"File downloaded successfully to temporary location: {temp_file_path}")

                fname=f"HDF4_EOS:EOS_GRID:{temp_file_path}:MOD_CMG_Snow_5km:Snow_Cover_Monthly_CMG"
                dataset = rxr.open_rasterio(fname)
                
                snowcover = dataset.isel(band=0) # Select the first band and plot it
                gdf_frame = gdf_frame.to_crs(snowcover.rio.crs)

                # Clip the raster to the GeoDataFrame
                clipped = snowcover.rio.clip(gdf_frame.geometry)
                clipped = clipped.where(clipped <= 250, np.nan)

                # Plot the clipped raster
                plt.figure(figsize=(10, 6))
                clipped.plot(cmap="viridis", vmin=0, vmax=100)
                plt.xlim(-180,-50)
                plt.ylim(0,80)
                plt.title(f"Monthly Snow Cover {dataset.attrs['RANGEBEGINNINGDATE']} to {dataset.attrs['RANGEENDINGDATE']}")
                figname = f"{figure_dir}/monthly_snow_cover_{dataset.attrs['RANGEBEGINNINGDATE']}_{dataset.attrs['RANGEENDINGDATE']}.png"
                plt.savefig(figname, dpi=300, bbox_inches='tight')
                plt.close()

                # Calculate mean snow cover for each frame_id
                mean_values = []

                # Suppress RuntimeWarnings temporarily
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", category=RuntimeWarning)
                    
                    for idx, row in gdf_frame.iterrows():
                        frame_id = row['frame_id']
                        geometry = row['geometry']
                                                
                        # Clip the raster to the current geometry
                        clipped = snowcover.rio.clip([geometry])
                        
                        # Remove pixels above 250
                        clipped = clipped.where(clipped <= 100, np.nan)
                        
                        # Calculate the mean of the clipped raster 
                        mean_value = np.nanmean(clipped).item()  # .item() extracts the scalar from the array
                        mean_values.append(mean_value)

                        # Plot a clipped raster
                        if frame_id==25018:
                            plt.figure(figsize=(10, 6))
                            clipped.plot(cmap="viridis", vmin=0, vmax=100)
                            plt.title(f"Frame ID {frame_id} ({mean_value:.2f} % Snow Cover) \n {dataset.attrs['RANGEBEGINNINGDATE']} to {dataset.attrs['RANGEENDINGDATE']}")
                            figname = f"{figure_dir}/F{frame_id}_{dataset.attrs['RANGEBEGINNINGDATE']}_{dataset.attrs['RANGEENDINGDATE']}.png"
                            plt.savefig(figname, dpi=300, bbox_inches='tight')
                            plt.close()

                # Append mean_values as a column in gdf_frames
                gdf_frame["mean_snowcover"] = mean_values

                # Replace nans to -1 so we can track later
                if hasattr(gdf_frame['mean_snowcover'], 'fillna'):
                    gdf_frame['mean_snowcover'] = gdf_frame['mean_snowcover'].fillna(-1)
                else:
                    gdf_frame['mean_snowcover'].fillna(-1, inplace=True)

                gdf_frame['to_process'] = gdf_frame['mean_snowcover'].apply(lambda x: 0 if x >= threshold_snowcover else 1)

                gdf_frame = gdf_frame.drop(['is_land', 'is_north_america', 'orbit_pass'], axis=1).reset_index(drop=True)
                gdf_frame.insert(1, 'year', year)
                gdf_frame.insert(2, 'month', month)

                return gdf_frame

            except requests.RequestException as e:
                print(f"Error downloading the file: {e}")
            except Exception as e:
                print(f"An error occurred: {e}")
            finally:
                # Clean up the temporary file
                if 'temp_file_path' in locals() and os.path.exists(temp_file_path):
                    os.unlink(temp_file_path)
                    # print(f"Cleaned up: Temporary file removed.")

    else:
        print(f"No data found for {year}-{month:02d}")
        return None

In [ ]:
def process_month(year, month, gdf_frames, threshold_snowcover):
    return get_snow_cover_data_filter(year, month, gdf_frames, threshold_snowcover)

In [ ]:
gdf_frames = opera_utils.get_frame_geojson(as_geodataframe=True)
if gdf_frames.index.name == 'frame_id':     
    gdf_frames = gdf_frames.reset_index()

gdf_frames = gdf_frames.loc[gdf_frames.is_north_america.isin([1, '1'])].reset_index(drop=True) 

In [ ]:
# Generate a list of (year, month) tuples for the date range
date_range = []
current_date = start_date
while current_date <= end_date:
    date_range.append((current_date.year, current_date.month))
    current_date += timedelta(days=32)
    current_date = current_date.replace(day=1)

# Process data for the entire date range in parallel with progress bar
all_gdf_frames = []
with ProcessPoolExecutor(max_workers=ncpus) as executor:
    futures = [executor.submit(process_month, year, month, gdf_frames, threshold_snowcover) for year, month in date_range]
    
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing months"):
        result = future.result()
        if result is not None:
            all_gdf_frames.append(result)

In [ ]:
merged_gdf = pd.concat(all_gdf_frames, ignore_index=True)
merged_gdf = merged_gdf.sort_values(by=['year', 'month', 'frame_id']).reset_index(drop=True)

In [ ]:
# Save the GeoDataFrame to a GeoJSON file
merged_gdf.to_file("snowcover_database.geojson", driver="GeoJSON")
merged_gdf

In [ ]:
# Add column that combies year and month in a datetime format
merged_gdf['date'] = pd.to_datetime(merged_gdf[['year', 'month']].assign(day=1))
merged_gdf

In [ ]:
# list frame_id with snow cover above threshold from merged_gdf
frameid_lst_snow = merged_gdf.loc[merged_gdf['mean_snowcover'] >= threshold_snowcover, 'frame_id'].unique()
frameid_lst_snow = list(set(frameid_lst_snow))
frameid_lst_snow = pd.DataFrame(frameid_lst_snow, columns=['frame_id'])
frameid_lst_snow.to_csv('frameid_lst_snow.csv', index=False)
frameid_lst_snow

In [ ]:
%matplotlib inline
plt.figure(figsize=(12, 4))
plt.plot(pd.to_datetime(merged_gdf.loc[(merged_gdf['frame_id']==25018) & (merged_gdf['mean_snowcover'] != -1)]['date']), merged_gdf.loc[(merged_gdf['frame_id']==25018) & (merged_gdf['mean_snowcover'] != -1)]['mean_snowcover'], marker='o', alpha=0.5, label='Fairbanks')
plt.plot(pd.to_datetime(merged_gdf.loc[(merged_gdf['frame_id']==4839) & (merged_gdf['mean_snowcover'] != -1)]['date']), merged_gdf.loc[(merged_gdf['frame_id']==4839) & (merged_gdf['mean_snowcover'] != -1)]['mean_snowcover'],  marker='o',  alpha=0.5, label='Panama')
plt.plot(pd.to_datetime(merged_gdf.loc[(merged_gdf['frame_id']==36545) & (merged_gdf['mean_snowcover'] != -1)]['date']), merged_gdf.loc[(merged_gdf['frame_id']==36545) & (merged_gdf['mean_snowcover'] != -1)]['mean_snowcover'],  marker='o', alpha=0.5, label='Sierra Nevada')
plt.plot(pd.to_datetime(merged_gdf.loc[(merged_gdf['frame_id']==11114) & (merged_gdf['mean_snowcover'] != -1)]['date']), merged_gdf.loc[(merged_gdf['frame_id']==11114) & (merged_gdf['mean_snowcover'] != -1)]['mean_snowcover'],  marker='o', alpha=0.5, label='Central Valley+Sierra Nevada')
plt.legend(loc='upper center', ncol=4)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Mean Snow Cover (%)', fontsize=20)
plt.ylim(-1, 111)
plt.xlim(datetime(2019, 6, 1), datetime(2021, 8, 1))
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)


In [ ]:
plt.figure(figsize=(15, 6))
merged_gdf.loc[merged_gdf['frame_id']==25018].groupby('month')['mean_snowcover'].apply(np.nanmean).replace(-1, np.nan).plot(label='Fairbanks', marker='o') 
merged_gdf.loc[merged_gdf['frame_id']==4839].groupby('month')['mean_snowcover'].apply(np.nanmean).replace(-1, np.nan).plot(label='Panama', marker='o') 
merged_gdf.loc[merged_gdf['frame_id']==36545].groupby('month')['mean_snowcover'].apply(np.nanmean).replace(-1, np.nan).plot(label='Sierra Nevada', marker='o') 
merged_gdf.loc[merged_gdf['frame_id']==11114].groupby('month')['mean_snowcover'].apply(np.nanmean).replace(-1, np.nan).plot(label='Central Valley + Sierra Nevada', marker='o') 
plt.legend(loc='upper center', ncol=4)
plt.xlabel('Month', fontsize=20)
plt.ylabel('Mean Snow Cover (%)', fontsize=20)
plt.ylim(-1, 111)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], fontsize=14)
plt.show

In [ ]:
frameid_lst_snow = []

# Plot the results as a function of month  
plt.figure(figsize=(15, 6))
for frame_id in merged_gdf['frame_id'].unique():
    mean_snowcover = merged_gdf.loc[merged_gdf['frame_id']==frame_id].groupby('month')['mean_snowcover'].apply(np.nanmean).replace(-1, np.nan)
    std_snowcover = merged_gdf.loc[merged_gdf['frame_id']==frame_id].groupby('month')['mean_snowcover'].apply(np.nanstd).replace(-1, np.nan)
    # plt.errorbar(mean_snowcover.index, mean_snowcover, yerr=std_snowcover, alpha=0.1, color='b')    
    plt.plot(mean_snowcover.index, mean_snowcover, color='b',alpha=0.05)
    # print(f"Frame ID: {frame_id}, Mean Snow Cover: {mean_snowcover}")
    if mean_snowcover.max()>=threshold_snowcover:
        frameid_lst_snow.append(frame_id)
plt.xlabel('Month', fontsize=20)
plt.ylabel('Mean Snow Cover (%)', fontsize=20)
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], fontsize=14)
plt.yticks(fontsize=14)

In [ ]:
frameid_lst_snow = list(set(frameid_lst_snow))
frameid_lst_snow = pd.DataFrame(frameid_lst_snow, columns=['frame_id'])
frameid_lst_snow.to_csv('frameid_lst_snow.csv', index=False)
frameid_lst_snow

In [ ]:
# mean of the mean_snowcover per month over the years. 
plt.figure(figsize=(15, 6))
for frame_id in merged_gdf['frame_id'].unique():
    mean_snowcover = merged_gdf.loc[merged_gdf['frame_id']==frame_id].groupby('month')['mean_snowcover'].apply(np.nanmean).replace(-1, np.nan)
    std_snowcover = merged_gdf.loc[merged_gdf['frame_id']==frame_id].groupby('month')['mean_snowcover'].apply(np.nanstd).replace(-1, np.nan)
    # plt.errorbar(mean_snowcover.index, mean_snowcover, yerr=std_snowcover, alpha=0.1, color='b')    
    plt.plot(mean_snowcover.index, mean_snowcover, color='b',alpha=0.05)
plt.xlabel('Month', fontsize=20)
plt.ylabel('Mean Snow Cover (%)', fontsize=20)
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], fontsize=14)
plt.yticks(fontsize=14)

In [ ]:
import geopandas as gpd

# New dataframe with the frame_id, mean_snowcover, and month
mean_snowcover_month = merged_gdf.groupby(['frame_id', 'month'])['mean_snowcover'].apply(np.nanmean).reset_index()

# Drop/Keep   
mean_snowcover_month['to_process'] = mean_snowcover_month['mean_snowcover'].apply(lambda x: 1 if x <= threshold_snowcover else 0)

# Add column geometry to mean_snowcover_month
mean_snowcover_month = mean_snowcover_month.merge(gdf_frames[['frame_id', 'geometry']], on='frame_id', how='left')

# Convert mean_snowcover_month to a geopandas dataframe
gdf_mean_snowcover_month = gpd.GeoDataFrame(mean_snowcover_month, geometry="geometry")
gdf_mean_snowcover_month

In [ ]:
# Center coordinates of the US
us_center = [37.0902, -95.7129]

# Plot to_process
gdf_mean_snowcover_month[gdf_mean_snowcover_month['month'] == 2][['frame_id', 'geometry', 'to_process']].explore(
    "to_process", 
    vmin=0,
    vmax=1,
    location=us_center, 
    zoom_start=3,
    cmap="RdBu",
)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Filter the GeoDataFrame for January (month == 1)
gdf_january = gdf_mean_snowcover_month[gdf_mean_snowcover_month['month'] == 1]

# Create a figure and axis with a proportional aspect ratio
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Plot the geometries using 'to_process' as the color value
cax = gdf_january.plot(
    column='to_process',
    cmap='RdBu',  # Colormap
    vmin=0,       # Minimum value for the color range
    vmax=1,       # Maximum value for the color range
    linewidth=0.8,
    edgecolor='black',
    legend=False,  # Disable default legend
    ax=ax,
    alpha=0.5
)

ax.set_xlim(-180, -50)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)

# Set the title
ax.set_title('Snow Cover in January ', fontsize=20)

# Set aspect ratio to be equal for proper proportional display
# ax.set_aspect('equal')

# Create a colorbar below the plot
divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.5)

# Add the colorbar with horizontal orientation
sm = plt.cm.ScalarMappable(cmap='RdBu', norm=plt.Normalize(vmin=0, vmax=1))
cbar = fig.colorbar(sm, cax=cax, orientation='horizontal')
cbar.set_ticks([0, 1])
cbar.ax.tick_params(labelsize=14)
cbar.set_label('Drop: 0 / Keep: 1', fontsize=20)


# Remove any extra whitespace
plt.tight_layout()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import calendar

# Define the list of months for iteration
months = gdf_mean_snowcover_month['month'].unique()

# Loop through each month and generate a plot
for month in months:
    # Filter the GeoDataFrame for the current month
    gdf_current_month = gdf_mean_snowcover_month[gdf_mean_snowcover_month['month'] == month]

    # Create a figure and axis with a proportional aspect ratio
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    # Plot the geometries using 'to_process' as the color value
    gdf_current_month.plot(
        column='to_process',
        cmap='RdBu',  # Colormap
        vmin=0,       # Minimum value for the color range
        vmax=1,       # Maximum value for the color range
        linewidth=0.8,
        edgecolor='black',
        legend=False,  # Disable default legend
        ax=ax,
        alpha=0.5
    )

    # Set axis limits (adjust to your region of interest)
    ax.set_xlim(-180, -50)

    # Set the tick labels to be larger
    ax.xaxis.set_tick_params(labelsize=14)
    ax.yaxis.set_tick_params(labelsize=14)

    # Get month name from month number
    month_name = calendar.month_name[month]

    # Set the title with the current month
    ax.set_title(f'{month_name}', fontsize=20)

    # Create a colorbar below the plot
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("bottom", size="5%", pad=0.5)

    # Add the colorbar with horizontal orientation
    sm = plt.cm.ScalarMappable(cmap='RdBu', norm=plt.Normalize(vmin=0, vmax=1))
    cbar = fig.colorbar(sm, cax=cax, orientation='horizontal')
    cbar.set_ticks([0, 1])
    cbar.ax.tick_params(labelsize=14)
    cbar.set_label('Drop: 0 / Keep: 1', fontsize=20)

    # Remove any extra whitespace
    plt.tight_layout()

    # Save the figure as a PNG file, with the month number in the filename
    plt.savefig(f'{figure_dir}/{month}_mean_meansnowcover.png', format='png')

    # Close the plot to free memory after saving
    plt.close()

In [ ]:
# plot the mean snow cover for each frame_id, for the entire date range, in a single plot, using gradient color for the frame_id.
plt.figure(figsize=(10, 6))
plt.scatter(merged_gdf['date'], merged_gdf['mean_snowcover'], c=merged_gdf['frame_id'], cmap='viridis')
plt.colorbar(label='Frame ID')
plt.xlabel('Date')
plt.ylabel('Mean Snow Cover (%)')
plt.title('Mean Snow Cover for Each Frame ID')

In [ ]:
#histogram of mean snow cover
plt.figure(figsize=(10, 6))
plt.hist(merged_gdf['mean_snowcover'], bins=10, color='skyblue', edgecolor='black', linewidth=1.2)
plt.xlabel('Mean Snow Cover (%)')
plt.ylabel('Frequency')

In [ ]:
# Center coordinates of the US
us_center = [37.0902, -95.7129]
year_selected = 2016
month_selected = 12

# Plot to_process
merged_gdf[(merged_gdf['year'] == year_selected) & (merged_gdf['month'] == month_selected)][['frame_id', 'geometry', 'to_process']].explore(
    "to_process", 
    vmin=0,
    vmax=1,
    location=us_center, 
    zoom_start=3,
    cmap="RdBu",
)

In [ ]:
year_selected = 2016
month_selected = 12
frameID_selected = 42779

# Plot to_process
merged_gdf[(merged_gdf['year'] == year_selected) & (merged_gdf['month'] == month_selected) & (merged_gdf['frame_id'] == frameID_selected)][['frame_id', 'geometry', 'to_process']].explore(
    "to_process", 
    vmin=0,
    vmax=1,
    cmap="RdBu",
)

In [ ]:
year_selected = 2016
month_selected = 12
frameID_selected = 33065

# Plot to_process
merged_gdf[(merged_gdf['year'] == year_selected) & (merged_gdf['month'] == month_selected) & (merged_gdf['frame_id'] == frameID_selected)][['frame_id', 'geometry', 'to_process']].explore(
    "to_process", 
    vmin=0,
    vmax=1,
    cmap="RdBu",
)

In [ ]:
# Export to geojson file
merged_gdf.to_file(output_geojson, driver="GeoJSON")